In [ ]:
!pip3 install retrying

import asyncio
import aiohttp
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nest_asyncio
from retrying import retry
import numpy as np

url ="https://www.moneycontrol.com/india/stockpricequote/finance-term-lending-institutions/powerfinancecorporation/PFC02"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"}

page=requests.get(url,headers)
soup=BeautifulSoup(page.text,'html.parser')

column_names=['Company','LTP']
for oview_div in soup.find('div',{'class':'nsestock_overview bsestock_overview'}).find_all('div',{'class':'oview_table'}):
  for row in oview_div.find('tbody').find_all('tr'):
    label=row.find_all('td')[0].text.strip()
    label='VWAP' if 'VWAP' in label else label
    column_names.append(label)


nest_asyncio.apply()

main_url = "https://www.moneycontrol.com/india/stockpricequote/"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36"
}

Data=[]

async def fetch_company_data(session, url):
  count=2
  d={}
  async with session.get(url, headers=headers) as response:
      html = await response.text()
      soup = BeautifulSoup(html, "html.parser")
      ltp_obj = soup.find("div", {"class": "inprice1 nsecp"})
      d[column_names[1]]=ltp_obj['rel']
      for oview_div in soup.find('div',{'class':'nsestock_overview bsestock_overview'}).find_all('div',{'class':'oview_table'}):
        for row in oview_div.find('tbody').find_all('tr'):
          value=row.find_all('td')[1].text.strip()
          if row.find_all('td')[0].text.strip()=='Beta':
            value=value.split('\n')[0]
          value=value.replace('\n','')
          value=value.replace(',','')
          value=float(value) if ('\n'not in value) and ('(' not in value) and ('-' not in value) else value
          d[column_names[count]]=value
          count+=1
      #oview_obj=soup.find_all("div",{"class":})
      return d

@retry(stop_max_attempt_number=3, wait_fixed=2000)  # Retry up to 3 times with a 2-second delay
async def fetch_with_retry(session, url):
  return await fetch_company_data(session, url)

async def main():
  async with aiohttp.ClientSession() as session:
      parentpage = await session.get(main_url)
      parent_soup = BeautifulSoup(await parentpage.text(), "html.parser")

      co_names = [a.get_text() for a in parent_soup.find_all("a", {"class": "bl_12"})]
      url_list = [link['href'] for link in parent_soup.find_all('a', class_='bl_12')]

      ltp_data = {}
      Data=[]

      for name,url in zip(co_names,url_list):
          try:
              d = await fetch_with_retry(session, url)
              d[column_names[0]]=name
              Data.append(d)
          except Exception as e:
              print(f"Error fetching data for URL: {url}, Error: {e}")
      return Data

all_data_list= asyncio.run(main())
df=pd.DataFrame(all_data_list)
df=df[column_names]

all_data_np = df.to_numpy()

In [ ]:
small_data_np=[i for i in all_data_np if float(i[1])<100 ]
med_data_np=[i for i in all_data_np if 100<float(i[1])<800 ]
large_data_np=[i for i in all_data_np if float(i[1])>800 ]


co_names=np.array([i[0] for i in all_data_np])
co_price=[float(i[1][0]) for i in all_data_np]

def makefile(arrays,filename):
  import csv
  csv_file_path=f"{filename}.csv"
  with open (csv_file_path,'w',newline='') as csv_file:
    writer =csv.writer(csv_file)
    writer.writerow(column_names)
    for i in arrays:
      writer.writerow(i)

makefile(small_data_np,'small')
makefile(med_data_np,'medium')
makefile(large_data_np,'large')
makefile(all_data_np,'all_data')